In [1]:
import pandas as pd

# Load each CSV file into a DataFrame
zerodha_df = pd.read_csv(r'C:\Users\umade\Downloads\Brokers_data\zerodha_script_master.csv').head(1000)
fyers_df = pd.read_csv(r'C:\Users\umade\Downloads\Brokers_data\fyers_script_master.csv').head(1000)
alice_df = pd.read_csv(r'C:\Users\umade\Downloads\Brokers_data\Alice_script_master.csv').head(1000)
samco_df = pd.read_csv(r'C:\Users\umade\Downloads\Brokers_data\samco_script_master.csv').head(1000)
angel_df = pd.read_csv(r'C:\Users\umade\Downloads\Brokers_data\angel_script_master.csv').head(1000)

C:\Users\umade\AppData\Local\Temp\ipykernel_1972\3442803196.py:8: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  angel_df = pd.read_csv(r'C:\Users\umade\Downloads\Brokers_data\angel_script_master.csv').head(1000)


In [2]:
angel_df

,token,Symbol,Instrument Name,Expiry Date,strike,lotsize,Instrument Type,exch_seg,tick_size
0,6293,GULPOLY-BL,GULPOLY,NaN,-1,1,NaN,NSE,1
1,24051,739UK30-SG,739UK30,NaN,-1,100,NaN,NSE,1
2,3475,THERMAX-EQ,THERMAX,NaN,-1,1,NaN,NSE,5
3,3523,69MZ33-SG,69MZ33,NaN,-1,100,NaN,NSE,1
4,17996,PGIMCSR2G-MF,PGIMCSR2G,NaN,-1,1,NaN,NSE,1
...,...,...,...,...,...,...,...,...,...
995,13976,TANLA-EQ,TANLA,NaN,-1,1,NaN,NSE,5
996,773,647MH28-SG,647MH28,NaN,-1,100,NaN,NSE,1
997,4067,MARICO-EQ,MARICO,NaN,-1,1,NaN,NSE,5
998,2112,MARALOVER-EQ,MARALOVER,NaN,-1,1,NaN,NSE,1


In [3]:
angel_df.keys()

Index(['token', 'Symbol', 'Instrument Name', 'Expiry Date', 'strike',
       'lotsize', 'Instrument Type', 'exch_seg', 'tick_size'],
      dtype='object')

In [4]:
fyers_df.keys()

Index(['Exch', 'Exchange Segment', 'Symbol', 'Token', 'Instrument Type',
       'Option Type', 'Strike Price', 'Instrument Name', 'Formatted Ins Name',
       'Trading Symbol', 'Expiry Date', 'Lot Size', 'Tick Size'],
      dtype='object')

In [5]:
# Standardize column names for each DataFrame
angel_df.rename(columns={
    'token': 'Token', 'Symbol': 'Symbol', 'Instrument Name': 'Instrument_Name',
    'Expiry Date': 'Expiry_Date', 'strike': 'Strike_Price', 'lotsize': 'Lot_Size',
    'Instrument Type': 'Instrument_Type', 'exch_seg': 'Exchange_Segment',
    'tick_size': 'Tick_Size'
}, inplace=True)

fyers_df.rename(columns={
    'Token': 'Token', 'Symbol': 'Symbol','Exchange Segment':'Exchange_Segment', 'Instrument Type': 'Instrument_Type',
    'Strike Price': 'Strike_Price', 'Instrument Name': 'Instrument_Name',
    'Formatted Ins Name': 'Formatted_Name', 'Trading Symbol': 'Trading_Symbol',
    'Expiry Date': 'Expiry_Date', 'Lot Size': 'Lot_Size', 'Tick Size': 'Tick_Size'
}, inplace=True)

alice_df.rename(columns={
    'Token': 'Token','Exchange Segment':'Exchange_Segment', 'Symbol': 'Symbol', 'Instrument Type': 'Instrument_Type',
    'Strike Price': 'Strike_Price', 'Instrument Name': 'Instrument_Name',
    'Formatted Ins Name': 'Formatted_Name', 'Trading Symbol': 'Trading_Symbol',
    'Expiry Date': 'Expiry_Date', 'Lot Size': 'Lot_Size', 'Tick Size': 'Tick_Size'
}, inplace=True)

samco_df.rename(columns={
    'symbolCode': 'Token', 'tradingSymbol': 'Symbol', 'name': 'Instrument_Name',
    'lotSize': 'Lot_Size', 'strikePrice': 'Strike_Price', 'expiryDate': 'Expiry_Date',
    'tickSize': 'Tick_Size', 'exchangeSegment': 'Exchange_Segment'
}, inplace=True)

zerodha_df.rename(columns={
    'instrument_token': 'Token', 'Symbol': 'Symbol', 'Instrument Name': 'Instrument_Name',
    'Expiry Date': 'Expiry_Date', 'strike': 'Strike_Price', 'tick_size': 'Tick_Size',
    'lot_size': 'Lot_Size', 'Instrument Type': 'Instrument_Type', 'segment': 'Exchange_Segment'
}, inplace=True)

In [6]:




# Merge data on common fields (Symbol and Exchange Segment)
# Using outer join to keep all records and identify missing values
merged_df = pd.merge(angel_df, fyers_df, on=['Symbol', 'Exchange_Segment'], suffixes=('_Angel', '_Fyers'), how='outer')
merged_df = pd.merge(merged_df, alice_df, on=['Symbol', 'Exchange_Segment'], suffixes=('', '_Alice'), how='outer')
merged_df = pd.merge(merged_df, samco_df, left_on=['Symbol'], right_on=['Symbol'], suffixes=('', '_Samco'), how='outer')
merged_df = pd.merge(merged_df, zerodha_df, left_on=['Symbol'], right_on=['Symbol'], suffixes=('', '_Zerodha'), how='outer')

# Drop any duplicate columns created from merging (optional)
# Drop columns like 'Expiry_Date_Alice', 'Strike_Price_Alice' if they are exact duplicates
for column in merged_df.columns:
    if "_Alice" in column or "_Samco" in column or "_Zerodha" in column or "_Fyers" in column or "_Angel" in column:
        # Drop the duplicate column if it matches another column without the suffix
        base_column = column.split('_')[0]
        if base_column in merged_df.columns and merged_df[column].equals(merged_df[base_column]):
            merged_df.drop(columns=[column], inplace=True)

# Fill missing values where possible, or leave as NaN for manual checking if necessary
# Here, we can fill in some default values or consolidate columns if required
merged_df['Lot_Size'] = merged_df['Lot_Size'].fillna(1)  # Example default fill
merged_df['Tick_Size'] = merged_df['Tick_Size'].fillna(0.01)  # Example default fill

# Save the merged DataFrame to a new CSV file
merged_df.to_csv("Unified_Script_Mapping.csv", index=False)

print("Unified CSV file created: Unified_Script_Mapping.csv")


Unified CSV file created: Unified_Script_Mapping.csv


In [7]:
alice_df.keys()

Index(['Exch', 'Exchange_Segment', 'Symbol', 'Token', 'Instrument_Type',
       'Option Type', 'Strike_Price', 'Instrument_Name', 'Formatted_Name',
       'Trading_Symbol', 'Expiry_Date', 'Lot_Size', 'Tick_Size'],
      dtype='object')